In [25]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

import matplotlib.pyplot as plt

In [26]:
# 1. Define the CNN model and two functions within it (one is constructor init, other is forward function)
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(32 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.relu = nn.ReLU()

        
    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = x.view(-1, 32 * 7 * 7)  # Flatten
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x
    

In [27]:
# 2. Load the dataset
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)
testloader = DataLoader(testset, batch_size=64, shuffle=False)

In [28]:
# 3. Initialize model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [30]:
# 4. Train the model
epochs = 5
for epoch in range(epochs):
    for images, labels in trainloader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        print(f"loss = {loss}, {type(loss)}")
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item():.4f}")

loss = 0.03471381217241287, <class 'torch.Tensor'>
loss = 0.28802865743637085, <class 'torch.Tensor'>
loss = 0.048630885779857635, <class 'torch.Tensor'>
loss = 0.08799388259649277, <class 'torch.Tensor'>
loss = 0.04938814043998718, <class 'torch.Tensor'>
loss = 0.011874177493155003, <class 'torch.Tensor'>
loss = 0.048095881938934326, <class 'torch.Tensor'>
loss = 0.02592962048947811, <class 'torch.Tensor'>
loss = 0.07284978032112122, <class 'torch.Tensor'>
loss = 0.031564127653837204, <class 'torch.Tensor'>
loss = 0.011180493980646133, <class 'torch.Tensor'>
loss = 0.03268185630440712, <class 'torch.Tensor'>
loss = 0.044064000248909, <class 'torch.Tensor'>
loss = 0.05751369148492813, <class 'torch.Tensor'>
loss = 0.0706566721200943, <class 'torch.Tensor'>
loss = 0.10485498607158661, <class 'torch.Tensor'>
loss = 0.16531279683113098, <class 'torch.Tensor'>
loss = 0.15033027529716492, <class 'torch.Tensor'>
loss = 0.023771753534674644, <class 'torch.Tensor'>
loss = 0.06897049397230148, 

In [ ]:
# 5. Evaluate the model
correct = 0
total = 0
model.eval()
with torch.no_grad():
    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(total, correct)

10000 9891


In [ ]:

print(f"Test Accuracy: {100 * correct / total:.2f}%")

Test Accuracy: 98.91%
